## This code corresponds to Example 3 of the [arXiv preprint](https://arxiv.org/abs/2404.13578)

It solves a simplified hemodynamics problem commonly employed as a benchmark to validate linear fluid-structure Interaction (FSI) solvers.

Tested with NGSolve version 6.2.2402

In [ ]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
import numpy as np

### Model coefficients and parameters

In [ ]:
t = Parameter(0.0)

tend = 12e-3 # end time
dt = 1e-4 # time step
    
#mass density
rhoF = 1
muF  = 0.035
lamF = 1e6
    
rhoS = 1.1
    
#Lamé coef. corresponding to C
muS  = 0.575e6
lamS = 1.7e6

beta = 4e6    
#needed for A = C**{-1}
aS1 = 0.5 / muS
aS2 = lamS / (4.0 * muS * (lamS + muS))
    
aF1 = 0.5 / muF
aF2 = lamF / (4.0 * muF * (lamF + muF))

pMax = 1.333e4
tMax = 0.003
pIn = IfPos(tMax - t, 0.5*pMax*( 1 - cos(2*np.pi*t/tMax))  , 0)

### Geometry and mesh

In [ ]:
geometry = SplineGeometry()
pnts     = [ (0, 0), (6,0), (6, 0.5), (6, 0.6), (0,0.6), (0, 0.5)]
pnums    = [geometry.AppendPoint(*p) for p in pnts]
#start-point, end-point, boundary-condition, domain on left side, domain on right side:
lines = [ (pnums[0], pnums[1], "gammaBot",  1, 0),
          (pnums[1], pnums[2], "gammaOut",  1, 0),
          (pnums[2], pnums[3], "gammaD",    2, 0),
          (pnums[3], pnums[4], "gammaN",    2, 0),
          (pnums[4], pnums[5], "gammaD",    2, 0),
          (pnums[5], pnums[0], "gammaIn",   1, 0),
          (pnums[5], pnums[2], "sigma",     2, 1)]

for p1, p2, bc, left, right in lines:
    if bc == "sigma":
        geometry.Append(["line", p1, p2], bc=bc, leftdomain=left, rightdomain=right, maxh=0.05)
    elif bc == "gammaN":
        geometry.Append(["line", p1, p2], bc=bc, leftdomain=left, rightdomain=right, maxh=0.05)
    else:
        geometry.Append(["line", p1, p2], bc=bc, leftdomain=left, rightdomain=right)

geometry.SetMaterial(1, "fluid")
geometry.SetMaterial(2, "solid")
        
h = 0.1 #mesh size
mesh = Mesh( geometry.GenerateMesh(maxh=h) )    

#mesh.GetMaterials()
#mesh.GetBoundaries()
Draw(mesh);

### Finite element spaces

In [ ]:
domain_values_rho = {'fluid': rhoF,  'solid': rhoS}
values_list_rho = [domain_values_rho[mat] for mat in mesh.GetMaterials()]
rho = CoefficientFunction(values_list_rho)

k = 2 #order of the finite element space

S = L2(mesh, order = k)
W = VectorL2(mesh, order = k+1)
D = VectorL2(mesh, order = k+1, definedon = "solid")
What0 = FacetFESpace(mesh, order=k+1, dirichlet="gammaD|gammaN")
What1 = FacetFESpace(mesh, order=k+1, dirichlet="gammaD|gammaIn|gammaBot")
fes = FESpace([S, S, S, W, D, What0, What1])

sigma1, sigma12, sigma2, u, d, uhat0, uhat1 = fes.TrialFunction()
tau1,   tau12,   tau2,   v, w, vhat0, vhat1 = fes.TestFunction()
    
sigma = CoefficientFunction(( sigma1, sigma12, sigma12, sigma2), dims = (2,2) )
tau   = CoefficientFunction(( tau1,   tau12,   tau12,   tau2),   dims = (2,2) )

uhat = CoefficientFunction(( uhat0, uhat1))
vhat = CoefficientFunction(( vhat0, vhat1))

AsigmaS = aS1 * sigma - aS2 * Trace(sigma) *  Id(mesh.dim)
AsigmaF = aF1 * sigma - aF2 * Trace(sigma) *  Id(mesh.dim)

jump_u = u - uhat
jump_v = v - vhat

### Bilinear forms and right-hand side

In [ ]:

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size
dS = dx(element_boundary=True)

a = BilinearForm(fes, condense=True)
a += (1/dt)*rho*u*v*dx
a += (1/dt)*InnerProduct(AsigmaS, tau)*dx("solid") 
a += (1/dt)*InnerProduct(d, w)*dx("solid")
a +=    0.5*beta*InnerProduct(d, v)*dx("solid")
a += -  0.5*InnerProduct(u,w)*dx("solid")
a +=    0.5*InnerProduct(AsigmaF,tau)*dx("fluid")
a +=    0.5*InnerProduct(sigma, grad(v))*dx    - 0.5*InnerProduct(grad(u), tau)*dx
a += -  0.5*InnerProduct( sigma*n, jump_v)*dS  + 0.5*InnerProduct(jump_u, tau*n)*dS
a +=    0.5*((k+1)**2/h)*jump_u*jump_v*dS
a.Assemble()

inv_A = a.mat.Inverse(freedofs=fes.FreeDofs(coupling=True))
    
M = BilinearForm(fes)
M += (1/dt)*rho*u*v*dx
M += (1/dt)*InnerProduct(AsigmaS, tau)*dx("solid")
M += (1/dt)*InnerProduct(d, w)*dx("solid")
M += - 0.5*beta*InnerProduct(d, v)*dx("solid")
M +=   0.5*InnerProduct(u,w)*dx("solid")
M += - 0.5*InnerProduct(AsigmaF,tau)*dx("fluid")
M += - 0.5*InnerProduct(sigma, grad(v))*dx   + 0.5*InnerProduct(grad(u), tau)*dx
M +=   0.5*InnerProduct( sigma*n, jump_v)*dS - 0.5*InnerProduct(jump_u, tau*n)*dS
M += - 0.5*((k+1)**2/h)*jump_u*jump_v*dS
M.Assemble()

ft = LinearForm(fes)
ft += -pIn*(vhat0.Trace()*n[0] + vhat1.Trace()*n[1])*ds(definedon=mesh.Boundaries("gammaIn"))

### Instantiation of initial conditions

In [ ]:

u0 = GridFunction(fes)
u0.vec[:] = 0.0

ft.Assemble()

res = u0.vec.CreateVector()
b0  = u0.vec.CreateVector()
b1  = u0.vec.CreateVector()
    
b0.data = ft.vec

t_intermediate = dt # time counter within one block-run

### Time loop

In [ ]:

while t_intermediate < tend:

    t.Set(t_intermediate)
    ft.Assemble()
    b1.data = ft.vec
     
    res.data = M.mat*u0.vec + 0.5*(b0.data + b1.data)
    
    u0.vec[:] = 0.0

    res.data = res - a.mat * u0.vec
    res.data += a.harmonic_extension_trans * res
    u0.vec.data += inv_A * res
    u0.vec.data += a.inner_solve * res
    u0.vec.data += a.harmonic_extension * u0.vec
    
    b0.data = b1.data
    
    t_intermediate += dt
    
    print("\r",t_intermediate,end="")

### Outputs corresponding to Figure 3 

In [ ]:
disp_values = []
velocity_values = []
pressure_values = []

start = 0
end = 6
step = 1/20
partition = np.arange(start, end+step, step)

for p in partition:
    disp_values.append(u0.components[4](mesh(p, 0.6))[1])
    velocity_values.append(2*u0.components[3](mesh(p, 0.0))[0]/3)
    pressure_values.append( - 0.5*u0.components[0](mesh(p, 0.0)) 
                            - 0.5*u0.components[2](mesh(p, 0.0)) )


fig, axs = plt.subplots(3)  # Create a figure with 4 subplots

# Plot disp_values
axs[0].plot(partition, disp_values)
axs[0].set_xlabel('top line')
axs[0].set_ylabel('y-disp')
axs[0].grid(True)

# Plot velocity_values
axs[1].plot(partition, velocity_values)
axs[1].set_xlabel('bottom line')
axs[1].set_ylabel('x-velocity')
axs[1].grid(True)

# Plot pressure_values
axs[2].plot(partition, pressure_values)
axs[2].set_xlabel('bottom line')
axs[2].set_ylabel('pressure')
axs[2].grid(True)

plt.tight_layout()  # Adjust the layout so that plots do not overlap
plt.show()